# metadata.csv 분석

In [1]:
# Repetition Counter 실험
import pandas as pd
import numpy as np
df = pd.read_csv("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_Repeatition_Counter/data/metadata.csv")

# 각 컬럼의 True/False 개수 세기
columns_to_check = ['frames_done', 'sapiens_done', 'reextract_done', 'overlay_done']

print("=" * 60)
for col in columns_to_check:
    print(f"\n{col}:")
    print("-" * 40)
    counts = df[col].value_counts()
    for value, count in counts.items():
        print(f"  {value}: {count}")
    print(f"  Total: {len(df)}")

print("\n" + "=" * 60)

# 요약 테이블
print("\n요약 테이블:")
print("-" * 60)
summary = pd.DataFrame({
    col: df[col].value_counts() for col in columns_to_check
}).T.fillna(0).astype(int)
print(summary)


frames_done:
----------------------------------------
  True: 1152
  Total: 1152

sapiens_done:
----------------------------------------
  True: 1152
  Total: 1152

reextract_done:
----------------------------------------
  True: 1152
  Total: 1152

overlay_done:
----------------------------------------
  True: 1152
  Total: 1152


요약 테이블:
------------------------------------------------------------
                True
frames_done     1152
sapiens_done    1152
reextract_done  1152
overlay_done    1152


# Keypoints 및 angle 좌표 시각화

In [2]:
import pandas as pd
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import logging

# 로깅 설정 (이전과 동일)
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')

# --- 1. 메타데이터 로드 및 JSON 로드 함수 (이전과 동일) ---

# 실제 파일 경로에 맞게 로드
df = pd.read_csv("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_Repeatition_Counter/data/metadata.csv")
train_df = df[df['is_train'] == True].copy()
logging.info(f"Total training samples: {len(train_df)}")

# JSON 로드 함수는 이전과 동일하게 유지합니다.
def load_keypoints_from_directory(json_dir):
    try:
        json_path = Path(json_dir)
        
        if not json_path.exists():
            logging.warning(f"Path does not exist: {json_dir}")
            return None
        
        json_files = sorted([f for f in json_path.glob('*.json')])
        
        if len(json_files) == 0:
            logging.warning(f"No JSON files found in: {json_dir}")
            return None
        
        keypoints_sequence = []
        for json_file in json_files:
            try:
                with open(json_file, 'r') as f:
                    frame_data = json.load(f)
                keypoints_sequence.append(frame_data)
            except Exception as e:
                logging.error(f"Error loading {json_file}: {e}")
                keypoints_sequence.append(None)
        
        logging.info(f"  Loaded {len(keypoints_sequence)} frames from {json_path.name}")
        return keypoints_sequence
        
    except Exception as e:
        logging.error(f"Error loading from {json_dir}: {e}")
        return None


# --- 2. Keypoints 좌표 추출 및 시각화 함수 (X-Y 좌표 동시 플롯) ---

def plot_filtered_keypoint_trajectory_xy(keypoints_sequence, sample_name):
    """
    Keypoint 인덱스 0~4를 제외한 관절의 X, Y 좌표 변화를 분리된 서브플롯으로 시각화합니다.
    """
    if not keypoints_sequence or not keypoints_sequence[0]:
        logging.warning(f"Cannot plot: keypoints sequence is empty or invalid for {sample_name}")
        return

    first_valid_frame = next((frame for frame in keypoints_sequence if frame and 'instance_info' in frame and len(frame['instance_info']) > 0), None)
    
    if not first_valid_frame or 'keypoints' not in first_valid_frame['instance_info'][0]:
        logging.warning(f"No valid keypoint data found in 'instance_info' for {sample_name}. Check JSON keys.")
        return

    num_keypoints = first_valid_frame['meta_info']['num_keypoints']
    keypoint_id2name = first_valid_frame['meta_info']['keypoint_id2name']
    
    # 🚨 필터링할 Keypoint 인덱스 (0, 1, 2, 3, 4)
    EXCLUDE_INDICES = set(range(5))
    
    trajectories = {i: {'x': [], 'y': []} for i in range(num_keypoints) if i not in EXCLUDE_INDICES}
    num_frames = len(keypoints_sequence)
    
    # 2-1. 프레임별 좌표 추출
    for frame_data in keypoints_sequence:
        keypoints_array = None
        if (frame_data and 'instance_info' in frame_data and 
            len(frame_data['instance_info']) > 0 and 
            'keypoints' in frame_data['instance_info'][0]):
            
            keypoints_list = frame_data['instance_info'][0]['keypoints']
            keypoints_array = np.array(keypoints_list) 

        for index in range(num_keypoints):
            if index in EXCLUDE_INDICES:
                continue

            if keypoints_array is not None and index < len(keypoints_array) and keypoints_array.ndim >= 2:
                # X (0), Y (1) 좌표를 추출
                trajectories[index]['x'].append(keypoints_array[index, 0])
                trajectories[index]['y'].append(keypoints_array[index, 1])
            else:
                trajectories[index]['x'].append(np.nan)
                trajectories[index]['y'].append(np.nan)

    # 2-2. 시각화 (X 및 Y 좌표)
    frames = np.arange(num_frames)
    
    # 2행 1열 서브플롯 생성
    fig, axes = plt.subplots(2, 1, figsize=(18, 12), sharex=True)
    fig.suptitle(f"Keypoint Trajectories (Filtered 5-16) for Sample: {sample_name}", fontsize=18)
    
    # X 좌표 (수평 이동) 플롯
    ax_x = axes[0]
    ax_x.set_title("X Coordinate Over Frames (Horizontal Movement)")
    for index, data in trajectories.items():
        keypoint_name = keypoint_id2name.get(str(index), f"KP_{index}")
        ax_x.plot(frames, data['x'], label=keypoint_name, alpha=0.7, linewidth=1.5)
        
    ax_x.set_ylabel("X Coordinate")
    ax_x.grid(True, linestyle='--', alpha=0.7)
    
    # Y 좌표 (수직 이동) 플롯
    ax_y = axes[1]
    ax_y.set_title("Y Coordinate Over Frames (Vertical Movement)")
    for index, data in trajectories.items():
        keypoint_name = keypoint_id2name.get(str(index), f"KP_{index}")
        ax_y.plot(frames, data['y'], label=keypoint_name, alpha=0.7, linewidth=1.5)
        
    ax_y.set_xlabel("Frame Number")
    ax_y.set_ylabel("Y Coordinate")
    ax_y.grid(True, linestyle='--', alpha=0.7)

    # 범례는 하나만 생성하고 전체 그림 외부에 배치
    handles, labels = ax_x.get_legend_handles_labels()
    fig.legend(handles, labels, loc='center right', bbox_to_anchor=(1.0, 0.5), ncol=1, fontsize=9)
    
    plt.tight_layout(rect=[0, 0, 0.9, 1]) # 범례를 위해 공간 확보
    plt.show()


# --- 3. 데이터프레임 순회 및 플롯 실행 (경로 기반 필터링) ---

TARGET_ACTIVITY_NAME = 'frontal__biceps_curl' 
TARGET_ACTIVITY_PATH_PATTERN = f'/{TARGET_ACTIVITY_NAME}'

# 경로 기반 필터링
target_df = train_df[train_df['interp_json_path'].str.contains(TARGET_ACTIVITY_PATH_PATTERN, case=False, na=False)].copy()

logging.info("-" * 50)
logging.info(f"Plotting trajectory for activity: {TARGET_ACTIVITY_NAME} (Found {len(target_df)} samples)")

NUM_SAMPLES_TO_PLOT = 1 

if len(target_df) == 0:
    logging.warning(f"No samples found for activity: {TARGET_ACTIVITY_NAME}")
else:
    for idx, row in target_df.head(NUM_SAMPLES_TO_PLOT).iterrows():
        json_dir = row['interp_json_path']
        sample_name = Path(json_dir).name
        
        logging.info(f"Analyzing sample: {sample_name}...")
        
        keypoints_sequence = load_keypoints_from_directory(json_dir)
        
        # 🚨 X-Y 좌표 동시 플롯 함수 호출
        plot_filtered_keypoint_trajectory_xy(keypoints_sequence, sample_name)

logging.info("-" * 50)
logging.info("Trajectory analysis complete.")

INFO - Total training samples: 83
INFO - --------------------------------------------------
INFO - Plotting trajectory for activity: frontal__biceps_curl (Found 3 samples)
INFO - Analyzing sample: frontal__biceps_curl...
WARNING - Path does not exist: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/4_INTERP_DATA/AI_dataset/N01/N01_Treatment/frontal__biceps_curl
WARNING - Cannot plot: keypoints sequence is empty or invalid for frontal__biceps_curl
INFO - --------------------------------------------------
INFO - Trajectory analysis complete.


In [3]:
import pandas as pd
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import logging

# 로깅 설정 (이전과 동일)
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')

# --- 1. 메타데이터 로드 및 JSON 로드 함수 (이전과 동일) ---

df = pd.read_csv("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_Repeatition_Counter/data/metadata.csv")
train_df = df[df['is_train'] == True].copy()
logging.info(f"Total training samples: {len(train_df)}")

# JSON 로드 함수는 이전과 동일하게 유지합니다.
def load_keypoints_from_directory(json_dir):
    try:
        json_path = Path(json_dir)
        # ... (중략: 파일 로드 로직) ...
        json_files = sorted([f for f in json_path.glob('*.json')])
        if len(json_files) == 0:
            logging.warning(f"No JSON files found in: {json_dir}")
            return None
        
        keypoints_sequence = []
        for json_file in json_files:
            try:
                with open(json_file, 'r') as f:
                    frame_data = json.load(f)
                keypoints_sequence.append(frame_data)
            except Exception as e:
                logging.error(f"Error loading {json_file}: {e}")
                keypoints_sequence.append(None)
        
        logging.info(f"  Loaded {len(keypoints_sequence)} frames from {json_path.name}")
        return keypoints_sequence
        
    except Exception as e:
        logging.error(f"Error loading from {json_dir}: {e}")
        return None


# --- 2. Keypoints 좌표 추출 및 궤적 플롯 함수 (Y축 반전 적용) ---

def plot_keypoint_path_xy_corrected(keypoints_sequence, sample_name):
    """
    Keypoint 5~16번의 X vs Y 궤적을 2차원 평면에 시각화합니다. (Y축 반전 적용)
    """
    if not keypoints_sequence or not keypoints_sequence[0]:
        logging.warning(f"Cannot plot: keypoints sequence is empty or invalid for {sample_name}")
        return

    first_valid_frame = next((frame for frame in keypoints_sequence if frame and 'instance_info' in frame and len(frame['instance_info']) > 0), None)
    
    if not first_valid_frame or 'keypoints' not in first_valid_frame['instance_info'][0]:
        logging.warning(f"No valid keypoint data found in 'instance_info' for {sample_name}. Check JSON keys.")
        return

    num_keypoints = first_valid_frame['meta_info']['num_keypoints']
    keypoint_id2name = first_valid_frame['meta_info']['keypoint_id2name']
    
    # 필터링할 Keypoint 인덱스 (0, 1, 2, 3, 4) 제외
    EXCLUDE_INDICES = set(range(5))
    
    trajectories = {i: {'x': [], 'y': []} for i in range(num_keypoints) if i not in EXCLUDE_INDICES}
    
    # 2-1. 프레임별 좌표 추출
    for frame_data in keypoints_sequence:
        keypoints_array = None
        if (frame_data and 'instance_info' in frame_data and 
            len(frame_data['instance_info']) > 0 and 
            'keypoints' in frame_data['instance_info'][0]):
            
            keypoints_list = frame_data['instance_info'][0]['keypoints']
            keypoints_array = np.array(keypoints_list) 

        for index in range(num_keypoints):
            if index in EXCLUDE_INDICES:
                continue

            if keypoints_array is not None and index < len(keypoints_array) and keypoints_array.ndim >= 2:
                trajectories[index]['x'].append(keypoints_array[index, 0])
                trajectories[index]['y'].append(keypoints_array[index, 1])
            else:
                trajectories[index]['x'].append(np.nan)
                trajectories[index]['y'].append(np.nan)

    # 2-2. 시각화 (X vs Y 궤적)
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.set_title(f"2D Keypoint Trajectory (X vs Y) - Corrected Y-axis\nSample: {sample_name}", fontsize=16)
    
    for index, data in trajectories.items():
        keypoint_name = keypoint_id2name.get(str(index), f"KP_{index}")
        
        # X vs Y 궤적을 라인 플롯으로 그립니다.
        ax.plot(data['x'], data['y'], 
                label=keypoint_name, 
                alpha=0.7, 
                linewidth=2.0)

    ax.set_xlabel("X Coordinate (Horizontal)")
    ax.set_ylabel("Y Coordinate (Vertical, Up is Positive)") 
    
    # 🚨 Y축 반전 적용: 이미지 좌표계를 일반적인 좌표계로 변환
    ax.invert_yaxis()
    
    # X와 Y 축 비율을 같게 만들어 궤적의 실제 모양을 보존합니다.
    ax.set_aspect('equal', adjustable='box') 
    
    ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.5), ncol=1, fontsize=10) 
    ax.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()


# --- 3. 데이터프레임 순회 및 플롯 실행 (경로 기반 필터링) ---

TARGET_ACTIVITY_NAME = 'frontal__biceps_curl' 
TARGET_ACTIVITY_PATH_PATTERN = f'/{TARGET_ACTIVITY_NAME}'

# 경로 기반 필터링
target_df = train_df[train_df['interp_json_path'].str.contains(TARGET_ACTIVITY_PATH_PATTERN, case=False, na=False)].copy()

logging.info("-" * 50)
logging.info(f"Plotting 2D Trajectory for activity: {TARGET_ACTIVITY_NAME} (Found {len(target_df)} samples)")

NUM_SAMPLES_TO_PLOT = 1 

if len(target_df) == 0:
    logging.warning(f"No samples found for activity: {TARGET_ACTIVITY_NAME}")
else:
    for idx, row in target_df.head(NUM_SAMPLES_TO_PLOT).iterrows():
        json_dir = row['interp_json_path']
        sample_name = Path(json_dir).name
        
        logging.info(f"Analyzing sample: {sample_name}...")
        
        keypoints_sequence = load_keypoints_from_directory(json_dir)
        
        # 🚨 수정된 궤적 플롯 함수 호출
        plot_keypoint_path_xy_corrected(keypoints_sequence, sample_name)

logging.info("-" * 50)
logging.info("Trajectory analysis complete.")

INFO - Total training samples: 83
INFO - --------------------------------------------------
INFO - Plotting 2D Trajectory for activity: frontal__biceps_curl (Found 3 samples)
INFO - Analyzing sample: frontal__biceps_curl...
WARNING - No JSON files found in: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/4_INTERP_DATA/AI_dataset/N01/N01_Treatment/frontal__biceps_curl
WARNING - Cannot plot: keypoints sequence is empty or invalid for frontal__biceps_curl
INFO - --------------------------------------------------
INFO - Trajectory analysis complete.


In [4]:
import pandas as pd
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import logging
import math

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')

# --- 0. 각도 계산 유틸리티 함수 ---

def calculate_angle(p1, p2, p3):
    """
    세 점 p1(start), p2(center), p3(end)을 이용하여 p2를 중심으로 하는 각도를 계산합니다.
    p1, p2, p3는 [x, y] 형태의 리스트/배열이어야 합니다.
    """
    try:
        p1 = np.array(p1)
        p2 = np.array(p2)
        p3 = np.array(p3)

        # 벡터 BA (p1 -> p2)
        # B를 중심으로 A와 C로 향하는 벡터를 사용해야 p2에서의 각도를 얻습니다.
        # 벡터 p2p1 (A)
        vec_a = p1 - p2
        # 벡터 p2p3 (C)
        vec_c = p3 - p2

        # 내적 (Dot Product)
        dot_product = np.dot(vec_a, vec_c)
        
        # 벡터 크기 (Norm)
        norm_a = np.linalg.norm(vec_a)
        norm_c = np.linalg.norm(vec_c)
        
        if norm_a == 0 or norm_c == 0:
            return np.nan # 길이가 0인 벡터는 계산 불가
        
        # 코사인 값 계산
        cosine_angle = dot_product / (norm_a * norm_c)
        
        # arccos 범위 제한 [-1, 1] (부동 소수점 오류 방지)
        cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
        
        # 라디안 -> 각도 (180도)
        angle_rad = np.arccos(cosine_angle)
        angle_deg = np.degrees(angle_rad)
        
        return angle_deg

    except Exception:
        return np.nan # 계산 중 오류 발생

# --- 1. 메타데이터 로드 및 JSON 로드 함수 (이전과 동일) ---

df = pd.read_csv("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_Repeatition_Counter/data/metadata.csv")
train_df = df[df['is_train'] == True].copy()
logging.info(f"Total training samples: {len(train_df)}")

# JSON 로드 함수는 이전과 동일하게 유지합니다.
def load_keypoints_from_directory(json_dir):
    # ... (이전 코드와 동일, 생략) ...
    try:
        json_path = Path(json_dir)
        # ... (중략) ...
        json_files = sorted([f for f in json_path.glob('*.json')])
        # ... (중략) ...
        
        keypoints_sequence = []
        for json_file in json_files:
            try:
                with open(json_file, 'r') as f:
                    frame_data = json.load(f)
                keypoints_sequence.append(frame_data)
            except Exception as e:
                logging.error(f"Error loading {json_file}: {e}")
                keypoints_sequence.append(None)
        
        logging.info(f"  Loaded {len(keypoints_sequence)} frames from {json_path.name}")
        return keypoints_sequence
        
    except Exception as e:
        logging.error(f"Error loading from {json_dir}: {e}")
        return None

# --- 2. 각도 추출 및 시각화 함수 ---

# Keypoint 인덱스 매핑 (p1, p2, p3)
ANGLE_DEFINITIONS = {
    "L_Elbow(5-7-9)":     (5, 7, 9),      # Shoulder - Elbow - Wrist
    "R_Elbow(6-8-10)":    (6, 8, 10),     # Shoulder - Elbow - Wrist
    "L_Shoulder(7-5-11)": (7, 5, 11),     # Elbow - Shoulder - Hip
    "R_Shoulder(8-6-12)": (8, 6, 12),     # Elbow - Shoulder - Hip
    "L_Hip_Joint(5-11-13)": (5, 11, 13),  # Shoulder - Hip - Knee (무릎 굴곡/신전 각도)
    "R_Hip_Joint(6-12-14)": (6, 12, 14),  # Shoulder - Hip - Knee
    "L_Knee(11-13-15)":   (11, 13, 15),   # Hip - Knee - Ankle
    "R_Knee(12-14-16)":   (12, 14, 16),   # Hip - Knee - Ankle
}


def plot_angle_trajectories(keypoints_sequence, sample_name):
    """
    8가지 주요 관절 각도의 변화를 시간에 따라 추적하여 시각화합니다.
    """
    if not keypoints_sequence or not keypoints_sequence[0]:
        logging.warning(f"Cannot plot: keypoints sequence is empty or invalid for {sample_name}")
        return

    first_valid_frame = next((frame for frame in keypoints_sequence if frame and 'instance_info' in frame and len(frame['instance_info']) > 0), None)
    if not first_valid_frame or 'keypoints' not in first_valid_frame['instance_info'][0]:
        logging.warning(f"No valid keypoint data found in 'instance_info' for {sample_name}.")
        return

    num_frames = len(keypoints_sequence)
    # 각도 데이터를 저장할 딕셔너리 초기화
    angle_trajectories = {name: [] for name in ANGLE_DEFINITIONS.keys()}
    
    # 2-1. 프레임별 각도 계산
    for frame_data in keypoints_sequence:
        keypoints_array = None
        current_kps = {} # {index: [x, y]}
        
        if (frame_data and 'instance_info' in frame_data and 
            len(frame_data['instance_info']) > 0 and 
            'keypoints' in frame_data['instance_info'][0]):
            
            keypoints_list = frame_data['instance_info'][0]['keypoints']
            keypoints_array = np.array(keypoints_list)
            
            # Keypoint를 {index: [x, y]} 형태로 저장
            for idx in range(len(keypoints_array)):
                # Keypoint 좌표는 (x, y)이고, 세 번째 요소는 신뢰도입니다.
                current_kps[idx] = keypoints_array[idx, :2] 

        # 8가지 각도에 대해 계산 및 저장
        for angle_name, (p1_idx, p2_idx, p3_idx) in ANGLE_DEFINITIONS.items():
            
            # 모든 점이 Keypoint 목록에 있어야 계산을 시도합니다.
            if keypoints_array is not None and all(idx in current_kps for idx in (p1_idx, p2_idx, p3_idx)):
                p1 = current_kps[p1_idx]
                p2 = current_kps[p2_idx] # 중심점
                p3 = current_kps[p3_idx]
                
                angle = calculate_angle(p1, p2, p3)
                angle_trajectories[angle_name].append(angle)
            else:
                angle_trajectories[angle_name].append(np.nan)

    # 2-2. 시각화
    frames = np.arange(num_frames)
    
    fig, ax = plt.subplots(figsize=(18, 8))
    ax.set_title(f"Joint Angle Trajectories over Time (Sample: {sample_name})", fontsize=16)
    
    # 각 각도 변화 플롯
    for angle_name, angles in angle_trajectories.items():
        # 바이셉스 컬이므로 팔꿈치 각도를 강조
        linewidth = 2.5 if 'Elbow' in angle_name else 1.5
        ax.plot(frames, angles, label=angle_name, linewidth=linewidth, alpha=0.8)
        
    ax.set_xlabel("Frame Number")
    ax.set_ylabel("Joint Angle (Degrees)")
    ax.set_ylim(0, 180) # 각도는 0도에서 180도 사이
    
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), ncol=1, fontsize=10) 
    ax.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()


# --- 3. 데이터프레임 순회 및 플롯 실행 (경로 기반 필터링) ---

TARGET_ACTIVITY_NAME = 'frontal__biceps_curl' 
TARGET_ACTIVITY_PATH_PATTERN = f'/{TARGET_ACTIVITY_NAME}'

# 경로 기반 필터링
target_df = train_df[train_df['interp_json_path'].str.contains(TARGET_ACTIVITY_PATH_PATTERN, case=False, na=False)].copy()

logging.info("-" * 50)
logging.info(f"Plotting Angle Trajectories for activity: {TARGET_ACTIVITY_NAME} (Found {len(target_df)} samples)")

NUM_SAMPLES_TO_PLOT = 1 

if len(target_df) == 0:
    logging.warning(f"No samples found for activity: {TARGET_ACTIVITY_NAME}")
else:
    for idx, row in target_df.head(NUM_SAMPLES_TO_PLOT).iterrows():
        json_dir = row['interp_json_path']
        sample_name = Path(json_dir).name
        
        logging.info(f"Analyzing sample: {sample_name}...")
        
        keypoints_sequence = load_keypoints_from_directory(json_dir)
        
        # 🚨 각도 추적 플롯 함수 호출
        plot_angle_trajectories(keypoints_sequence, sample_name)

logging.info("-" * 50)
logging.info("Trajectory analysis complete.")

INFO - Total training samples: 83
INFO - --------------------------------------------------
INFO - Plotting Angle Trajectories for activity: frontal__biceps_curl (Found 3 samples)
INFO - Analyzing sample: frontal__biceps_curl...
INFO -   Loaded 0 frames from frontal__biceps_curl
WARNING - Cannot plot: keypoints sequence is empty or invalid for frontal__biceps_curl
INFO - --------------------------------------------------
INFO - Trajectory analysis complete.


# repetition counter

In [5]:
import pandas as pd
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import logging
from scipy.signal import find_peaks
from scipy.ndimage import uniform_filter1d

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')

# --- 1. 메타데이터 로드 및 JSON 로드 함수 ---

df = pd.read_csv("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_Repeatition_Counter/data/metadata.csv")
train_df = df[df['is_train'] == True].copy()
logging.info(f"Total training samples: {len(train_df)}")

def load_keypoints_from_directory(json_dir):
    """JSON 디렉토리에서 키포인트 시퀀스를 로드합니다."""
    try:
        json_path = Path(json_dir)
        json_files = sorted([f for f in json_path.glob('*.json')])
        if len(json_files) == 0:
            logging.warning(f"No JSON files found in: {json_dir}")
            return None
        
        keypoints_sequence = []
        for json_file in json_files:
            try:
                with open(json_file, 'r') as f:
                    frame_data = json.load(f)
                keypoints_sequence.append(frame_data)
            except Exception as e:
                logging.error(f"Error loading {json_file}: {e}")
                keypoints_sequence.append(None)
        
        logging.info(f"  Loaded {len(keypoints_sequence)} frames from {json_path.name}")
        return keypoints_sequence
        
    except Exception as e:
        logging.error(f"Error loading from {json_dir}: {e}")
        return None


# --- 2. 단계 1: 운동별 핵심 관절 자동 선택 (분산 기반) ---
def extract_all_trajectories(keypoints_sequence, exclude_indices=None):
    """
    모든 Keypoint의 X, Y 좌표 궤적을 추출합니다.
    
    Returns:
        dict: {keypoint_index: {'x': [...], 'y': [...], 'keypoint_name': str}}
    """
    if not keypoints_sequence or not keypoints_sequence[0]:
        return {}
    
    first_valid_frame = next((frame for frame in keypoints_sequence 
                             if frame and 'instance_info' in frame 
                             and len(frame['instance_info']) > 0), None)
    
    if not first_valid_frame:
        return {}
    
    num_keypoints = first_valid_frame['meta_info']['num_keypoints']
    keypoint_id2name = first_valid_frame['meta_info']['keypoint_id2name']
    
    if exclude_indices is None:
        exclude_indices = set(range(5))  # 기본적으로 0~4번 제외
    
    trajectories = {}
    for i in range(num_keypoints):
        if i not in exclude_indices:
            trajectories[i] = {
                'x': [],
                'y': [],
                'keypoint_name': keypoint_id2name.get(str(i), f"KP_{i}")
            }
    
    # 프레임별 좌표 추출
    for frame_data in keypoints_sequence:
        keypoints_array = None
        if (frame_data and 'instance_info' in frame_data and 
            len(frame_data['instance_info']) > 0 and 
            'keypoints' in frame_data['instance_info'][0]):
            
            keypoints_list = frame_data['instance_info'][0]['keypoints']
            keypoints_array = np.array(keypoints_list)
        
        for index in trajectories.keys():
            if keypoints_array is not None and index < len(keypoints_array) and keypoints_array.ndim >= 2:
                trajectories[index]['x'].append(keypoints_array[index, 0])
                trajectories[index]['y'].append(keypoints_array[index, 1])
            else:
                trajectories[index]['x'].append(np.nan)
                trajectories[index]['y'].append(np.nan)
    
    return trajectories


def calculate_keypoint_variance(trajectories):
    """
    각 Keypoint의 X, Y 궤적에 대한 분산을 계산합니다.
    
    Returns:
        dict: {keypoint_index: variance_value}
    """
    variances = {}
    
    for kp_idx, data in trajectories.items():
        x_arr = np.array(data['x'])
        y_arr = np.array(data['y'])
        
        # NaN 제거
        x_valid = x_arr[~np.isnan(x_arr)]
        y_valid = y_arr[~np.isnan(y_arr)]
        
        if len(x_valid) > 0 and len(y_valid) > 0:
            var_x = np.var(x_valid)
            var_y = np.var(y_valid)
            variances[kp_idx] = var_x + var_y
        else:
            variances[kp_idx] = 0.0
    
    return variances


def select_core_joint(variances, trajectories, top_k=3):
    """
    분산이 가장 큰 Keypoint들을 핵심 관절로 선택합니다.
    
    Returns:
        list: [(keypoint_index, variance, keypoint_name), ...]
    """
    sorted_kps = sorted(variances.items(), key=lambda x: x[1], reverse=True)
    
    core_joints = []
    for kp_idx, var_value in sorted_kps[:top_k]:
        kp_name = trajectories[kp_idx]['keypoint_name']
        core_joints.append((kp_idx, var_value, kp_name))
    
    return core_joints


# --- 3. 단계 2: 관절 각도 계산 함수 ---

def calculate_angle(p1, p2, p3):
    """
    세 점으로 이루어진 각도를 계산합니다 (p2가 꼭지점).
    
    Returns:
        float: 각도 (degree)
    """
    v1 = np.array([p1[0] - p2[0], p1[1] - p2[1]])
    v2 = np.array([p3[0] - p2[0], p3[1] - p2[1]])
    
    cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-8)
    cos_angle = np.clip(cos_angle, -1.0, 1.0)
    angle = np.arccos(cos_angle)
    
    return np.degrees(angle)


def extract_joint_angles(keypoints_sequence, joint_config):
    """
    특정 관절의 각도 시계열을 계산합니다.
    
    Args:
        joint_config: dict with 'joint_idx', 'parent_idx', 'child_idx'
    
    Returns:
        np.array: 각도 시계열
    """
    angles = []
    
    for frame_data in keypoints_sequence:
        if (frame_data and 'instance_info' in frame_data and 
            len(frame_data['instance_info']) > 0 and 
            'keypoints' in frame_data['instance_info'][0]):
            
            keypoints_array = np.array(frame_data['instance_info'][0]['keypoints'])
            
            try:
                p_parent = keypoints_array[joint_config['parent_idx']][:2]
                p_joint = keypoints_array[joint_config['joint_idx']][:2]
                p_child = keypoints_array[joint_config['child_idx']][:2]
                
                angle = calculate_angle(p_parent, p_joint, p_child)
                angles.append(angle)
            except:
                angles.append(np.nan)
        else:
            angles.append(np.nan)
    
    return np.array(angles)


# --- 4. 단계 3: Peak/Valley 탐지 및 필터링 ---

def smooth_signal(signal, window_size=5):
    """이동 평균 필터를 사용하여 신호를 스무딩합니다."""
    valid_mask = ~np.isnan(signal)
    smoothed = signal.copy()
    
    if np.sum(valid_mask) > window_size:
        smoothed[valid_mask] = uniform_filter1d(signal[valid_mask], size=window_size, mode='nearest')
    
    return smoothed


def detect_peaks_valleys(signal, prominence_ratio=0.15, distance=10):
    """
    Peak와 Valley를 탐지합니다.
    
    Args:
        prominence_ratio: 신호 범위의 몇 %를 prominence로 사용할지 (0.15 = 15%)
        distance: Peak 간 최소 거리 (프레임 단위)
    
    Returns:
        peaks, valleys: Peak와 Valley의 인덱스 배열
    """
    valid_mask = ~np.isnan(signal)
    valid_signal = signal[valid_mask]
    
    if len(valid_signal) < 10:
        return np.array([]), np.array([])
    
    # 신호 범위 기반 prominence 계산
    signal_range = np.max(valid_signal) - np.min(valid_signal)
    prominence = signal_range * prominence_ratio
    
    # Peak 탐지
    peaks, _ = find_peaks(valid_signal, prominence=prominence, distance=distance)
    
    # Valley 탐지 (신호 반전)
    valleys, _ = find_peaks(-valid_signal, prominence=prominence, distance=distance)
    
    # 원래 인덱스로 매핑
    valid_indices = np.where(valid_mask)[0]
    peaks = valid_indices[peaks]
    valleys = valid_indices[valleys]
    
    return peaks, valleys


# --- 5. 단계 4: 반복 횟수 계산 ---

def count_repetitions(peaks, valleys):
    """
    Peak와 Valley의 교차를 기반으로 반복 횟수를 계산합니다.
    
    Returns:
        int: 반복 횟수
    """
    if len(peaks) == 0 or len(valleys) == 0:
        return 0
    
    # Peak와 Valley를 시간 순서대로 병합
    events = []
    for p in peaks:
        events.append((p, 'peak'))
    for v in valleys:
        events.append((v, 'valley'))
    
    events.sort(key=lambda x: x[0])
    
    # 교차 패턴 탐지
    rep_count = 0
    last_event = None
    
    for idx, event_type in events:
        if last_event is not None and last_event != event_type:
            rep_count += 1
        last_event = event_type
    
    # 한 번의 교차는 반복의 절반이므로 2로 나눔
    return rep_count // 2


# --- 6. 통합 함수: 일반화된 Repetition Counter ---

def generalized_repetition_counter(keypoints_sequence, sample_name, 
                                   use_angle=True, 
                                   prominence_ratio=0.15,
                                   window_size=5,
                                   visualize=True):
    """
    일반화된 반복 횟수 카운터.
    
    Args:
        keypoints_sequence: 키포인트 시퀀스
        sample_name: 샘플 이름
        use_angle: True면 관절 각도 사용, False면 Y 좌표 사용
        prominence_ratio: Peak 탐지 민감도 (0.1~0.2 권장)
        window_size: 스무딩 윈도우 크기
        visualize: 시각화 여부
    
    Returns:
        dict: 결과 정보
    """
    logging.info(f"\n{'='*60}")
    logging.info(f"Analyzing: {sample_name}")
    logging.info(f"{'='*60}")
    
    # 단계 1: 모든 Keypoint 궤적 추출 및 분산 계산
    trajectories = extract_all_trajectories(keypoints_sequence)
    if not trajectories:
        logging.error("No valid trajectories found!")
        return None
    
    variances = calculate_keypoint_variance(trajectories)
    core_joints = select_core_joint(variances, trajectories, top_k=5)
    
    logging.info("\n[Step 1] Top 5 Core Joints (Highest Variance):")
    for rank, (kp_idx, var_val, kp_name) in enumerate(core_joints, 1):
        logging.info(f"  {rank}. {kp_name} (KP {kp_idx}): Variance = {var_val:.2f}")
    
    # 가장 분산이 큰 관절 선택
    primary_joint_idx, _, primary_joint_name = core_joints[0]
    logging.info(f"\n[Step 2] Selected Primary Joint: {primary_joint_name} (KP {primary_joint_idx})")
    
    # 단계 2: 신호 추출 (각도 또는 좌표)
    if use_angle:
        # 관절 각도 계산을 위한 설정 (예시: 팔꿈치, 무릎 등)
        # 실제로는 관절 타입에 따라 parent, child를 동적으로 결정해야 함
        # 여기서는 간단히 인접 관절을 사용
        joint_config = {
            'joint_idx': primary_joint_idx,
            'parent_idx': max(5, primary_joint_idx - 2),
            'child_idx': min(16, primary_joint_idx + 2)
        }
        signal = extract_joint_angles(keypoints_sequence, joint_config)
        signal_name = "Joint Angle"
    else:
        # Y 좌표 사용
        signal = np.array(trajectories[primary_joint_idx]['y'])
        signal_name = "Y Coordinate"
    
    logging.info(f"  Tracking: {signal_name}")
    
    # 단계 3: 스무딩 및 Peak/Valley 탐지
    smoothed_signal = smooth_signal(signal, window_size=window_size)
    peaks, valleys = detect_peaks_valleys(smoothed_signal, 
                                         prominence_ratio=prominence_ratio,
                                         distance=10)
    
    logging.info(f"\n[Step 3] Peak/Valley Detection:")
    logging.info(f"  Found {len(peaks)} peaks and {len(valleys)} valleys")
    
    # 단계 4: 반복 횟수 계산
    rep_count = count_repetitions(peaks, valleys)
    logging.info(f"\n[Step 4] Repetition Count: {rep_count}")
    
    # 시각화
    if visualize:
        fig, axes = plt.subplots(2, 1, figsize=(14, 10))
        
        # 상단: 2D 궤적
        ax1 = axes[0]
        ax1.plot(trajectories[primary_joint_idx]['x'], 
                trajectories[primary_joint_idx]['y'], 
                'b-', linewidth=2, label=primary_joint_name)
        ax1.set_xlabel("X Coordinate")
        ax1.set_ylabel("Y Coordinate")
        ax1.set_title(f"2D Trajectory: {primary_joint_name}")
        ax1.invert_yaxis()
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        ax1.set_aspect('equal', adjustable='box')
        
        # 하단: 시계열 신호 + Peak/Valley
        ax2 = axes[1]
        frames = np.arange(len(signal))
        ax2.plot(frames, signal, 'b-', alpha=0.3, label='Original', linewidth=1)
        ax2.plot(frames, smoothed_signal, 'b-', label='Smoothed', linewidth=2)
        ax2.plot(peaks, smoothed_signal[peaks], 'ro', markersize=10, label='Peaks')
        ax2.plot(valleys, smoothed_signal[valleys], 'go', markersize=10, label='Valleys')
        
        ax2.set_xlabel("Frame")
        ax2.set_ylabel(signal_name)
        ax2.set_title(f"{signal_name} vs Time | Repetitions: {rep_count}")
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        
        plt.suptitle(f"Generalized Repetition Counter\nSample: {sample_name}", 
                    fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
    
    result = {
        'sample_name': sample_name,
        'primary_joint': primary_joint_name,
        'primary_joint_idx': primary_joint_idx,
        'variance': variances[primary_joint_idx],
        'num_peaks': len(peaks),
        'num_valleys': len(valleys),
        'repetition_count': rep_count,
        'signal_type': signal_name
    }
    
    return result


# --- 7. 실행 및 테스트 ---

if __name__ == "__main__":
    # 테스트할 운동 선택
    TARGET_ACTIVITY_NAME = 'frontal__biceps_curl'  # 또는 다른 운동으로 변경 가능
    TARGET_ACTIVITY_PATH_PATTERN = f'/{TARGET_ACTIVITY_NAME}'
    
    # 경로 기반 필터링
    target_df = train_df[train_df['interp_json_path'].str.contains(
        TARGET_ACTIVITY_PATH_PATTERN, case=False, na=False)].copy()
    
    logging.info(f"\n{'='*60}")
    logging.info(f"Testing Generalized Repetition Counter")
    logging.info(f"Activity: {TARGET_ACTIVITY_NAME}")
    logging.info(f"Found {len(target_df)} samples")
    logging.info(f"{'='*60}")
    
    NUM_SAMPLES_TO_TEST = 3  # 테스트할 샘플 수
    
    if len(target_df) == 0:
        logging.warning(f"No samples found for activity: {TARGET_ACTIVITY_NAME}")
    else:
        results = []
        
        for idx, row in target_df.head(NUM_SAMPLES_TO_TEST).iterrows():
            json_dir = row['interp_json_path']
            sample_name = Path(json_dir).name
            
            keypoints_sequence = load_keypoints_from_directory(json_dir)
            
            if keypoints_sequence:
                result = generalized_repetition_counter(
                    keypoints_sequence, 
                    sample_name,
                    use_angle=False,  # Y 좌표 사용 (간단한 테스트)
                    prominence_ratio=0.15,
                    window_size=7,
                    visualize=True
                )
                
                if result:
                    results.append(result)
        
        # 결과 요약
        if results:
            logging.info(f"\n{'='*60}")
            logging.info("SUMMARY")
            logging.info(f"{'='*60}")
            for r in results:
                logging.info(f"\nSample: {r['sample_name']}")
                logging.info(f"  Primary Joint: {r['primary_joint']} (KP {r['primary_joint_idx']})")
                logging.info(f"  Variance: {r['variance']:.2f}")
                logging.info(f"  Peaks/Valleys: {r['num_peaks']}/{r['num_valleys']}")
                logging.info(f"  Repetition Count: {r['repetition_count']}")

# 원하는 운동으로 변경 가능
TARGET_ACTIVITY_NAME = 'frontal__biceps_curl'

# 파라미터 조정 가능
result = generalized_repetition_counter(
    keypoints_sequence, 
    sample_name,
    use_angle=False,      # True: 관절 각도, False: Y 좌표
    prominence_ratio=0.15, # 0.1~0.2 권장 (민감도 조절)
    window_size=7,        # 스무딩 강도
    visualize=True        # 시각화 여부
)

INFO - Total training samples: 83
INFO - 
INFO - Testing Generalized Repetition Counter
INFO - Activity: frontal__biceps_curl
INFO - Found 3 samples
INFO - ============================================================
WARNING - No JSON files found in: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/4_INTERP_DATA/AI_dataset/N01/N01_Treatment/frontal__biceps_curl
WARNING - No JSON files found in: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/4_INTERP_DATA/AI_dataset/N01/N01_Ward/frontal__biceps_curl
WARNING - No JSON files found in: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/4_INTERP_DATA/AI_dataset/N01/N01_Ward/frontal__biceps_curlbilateral
INFO - 
INFO - Analyzing: frontal__biceps_curlbilateral
INFO - ============================================================
ERROR - No valid trajectories found!


## threshold 방식

In [6]:
import pandas as pd
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import logging
import math
from scipy.ndimage import uniform_filter1d # 이동 평균 필터 사용을 위해 import

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')

# --- 0. 각도 계산 유틸리티 함수 (이전과 동일) ---

def calculate_angle(p1, p2, p3):
    # ... (이전 코드의 calculate_angle 함수 내용) ...
    try:
        p1 = np.array(p1)
        p2 = np.array(p2)
        p3 = np.array(p3)
        vec_a = p1 - p2
        vec_c = p3 - p2
        dot_product = np.dot(vec_a, vec_c)
        norm_a = np.linalg.norm(vec_a)
        norm_c = np.linalg.norm(vec_c)
        
        if norm_a == 0 or norm_c == 0:
            return np.nan
        
        cosine_angle = dot_product / (norm_a * norm_c)
        cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
        angle_deg = np.degrees(np.arccos(cosine_angle))
        
        return angle_deg
    except Exception:
        return np.nan

# Keypoint 인덱스 매핑 (p1, p2, p3)
# 분석할 각도만 정의 (바이스텝스 컬의 핵심: 팔꿈치)
ANGLE_DEFINITIONS = {
    "L_Elbow(5-7-9)": (5, 7, 9),      # Shoulder - Elbow - Wrist
    "R_Elbow(6-8-10)": (6, 8, 10),     # Shoulder - Elbow - Wrist
}

# --- 1. 메타데이터 로드 및 JSON 로드 함수 (이전과 동일) ---

df = pd.read_csv("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_Repeatition_Counter/data/metadata.csv")
train_df = df[df['is_train'] == True].copy()
logging.info(f"Total training samples: {len(train_df)}")

def load_keypoints_from_directory(json_dir):
    # ... (이전 코드와 동일, 생략) ...
    try:
        json_path = Path(json_dir)
        # ... (중략) ...
        json_files = sorted([f for f in json_path.glob('*.json')])
        # ... (중략) ...
        
        keypoints_sequence = []
        for json_file in json_files:
            try:
                with open(json_file, 'r') as f:
                    frame_data = json.load(f)
                keypoints_sequence.append(frame_data)
            except Exception as e:
                logging.error(f"Error loading {json_file}: {e}")
                keypoints_sequence.append(None)
        
        logging.info(f"  Loaded {len(keypoints_sequence)} frames from {json_path.name}")
        return keypoints_sequence
        
    except Exception as e:
        logging.error(f"Error loading from {json_dir}: {e}")
        return None

# --- 2. Threshold 기반 카운팅 및 시각화 함수 ---

def plot_and_count_with_threshold(keypoints_sequence, sample_name):
    
    # ... (각도 추출 로직) ...
    if not keypoints_sequence or not keypoints_sequence[0]:
        logging.warning(f"Cannot count: keypoints sequence is empty or invalid for {sample_name}")
        return

    first_valid_frame = next((frame for frame in keypoints_sequence if frame and 'instance_info' in frame and len(frame['instance_info']) > 0), None)
    if not first_valid_frame or 'keypoints' not in first_valid_frame['instance_info'][0]:
        logging.warning(f"No valid keypoint data found in 'instance_info' for {sample_name}.")
        return

    num_frames = len(keypoints_sequence)
    angle_trajectories = {name: [] for name in ANGLE_DEFINITIONS.keys()}
    
    # 2-1. 프레임별 각도 계산 (이전 코드와 동일)
    for frame_data in keypoints_sequence:
        keypoints_array = None
        current_kps = {}
        
        if (frame_data and 'instance_info' in frame_data and 
            len(frame_data['instance_info']) > 0 and 
            'keypoints' in frame_data['instance_info'][0]):
            
            keypoints_list = frame_data['instance_info'][0]['keypoints']
            keypoints_array = np.array(keypoints_list)
            
            for idx in range(len(keypoints_array)):
                current_kps[idx] = keypoints_array[idx, :2] 

        for angle_name, (p1_idx, p2_idx, p3_idx) in ANGLE_DEFINITIONS.items():
            if keypoints_array is not None and all(idx in current_kps for idx in (p1_idx, p2_idx, p3_idx)):
                angle = calculate_angle(current_kps[p1_idx], current_kps[p2_idx], current_kps[p3_idx])
                angle_trajectories[angle_name].append(angle)
            else:
                angle_trajectories[angle_name].append(np.nan)

    # 🚨 Left Elbow Angle (왼쪽 팔꿈치 각도)만 분석 대상으로 선택
    if 'L_Elbow(5-7-9)' not in angle_trajectories:
        logging.warning("L_Elbow data not found in trajectories.")
        return
        
    raw_angles = np.array(angle_trajectories['L_Elbow(5-7-9)'])
    
    # 2-2. 데이터 전처리 (Moving Average Filter)
    # 윈도우 크기 설정 (예: 5 프레임)
    window_size = 5
    smoothed_angles = uniform_filter1d(raw_angles, size=window_size, mode='nearest')
    
    # 2-3. Threshold 및 State Machine 설정
    
    # 운동에 맞게 조정 필요: 바이셉스 컬은 170도(폄)에서 30도(굽힘) 사이의 값을 가집니다.
    UPPER_THRESHOLD = 150 # 팔을 편 상태 (DOWN) 임계값
    LOWER_THRESHOLD = 50  # 팔을 굽힌 상태 (UP) 임계값

    # State 정의: 0=DOWN, 1=UP
    current_state = 0 
    rep_count = 0
    
    # 카운팅 로직을 시각화하기 위한 이벤트 리스트
    count_events = [] 

    # 2-4. 반복 횟수 카운팅
    for frame_idx, angle in enumerate(smoothed_angles):
        if np.isnan(angle):
            continue

        # 상태 전환 조건 확인
        if current_state == 0: # 현재 상태: DOWN (팔을 편 상태)
            if angle < LOWER_THRESHOLD:
                # DOWN -> UP 전환 (팔 굽힘 시작 -> 수축 완료)
                current_state = 1
                count_events.append((frame_idx, angle, 'UP'))
                
        elif current_state == 1: # 현재 상태: UP (팔을 굽힌 상태)
            if angle > UPPER_THRESHOLD:
                # UP -> DOWN 전환 (수축 완료 -> 복귀 완료)
                rep_count += 1
                current_state = 0
                count_events.append((frame_idx, angle, 'DOWN'))

    # 2-5. 시각화
    frames = np.arange(num_frames)
    fig, ax = plt.subplots(figsize=(18, 6))
    
    ax.set_title(f"Threshold-based Repetition Counter for L_Elbow (Sample: {sample_name})\nTotal Reps: {rep_count}", fontsize=16)
    
    # 스무딩된 각도 플롯
    ax.plot(frames, smoothed_angles, label='Smoothed L_Elbow Angle', color='blue', linewidth=2)
    
    # Threshold 라인
    ax.axhline(UPPER_THRESHOLD, color='red', linestyle='--', label=f'Upper Threshold ({UPPER_THRESHOLD}°)')
    ax.axhline(LOWER_THRESHOLD, color='green', linestyle='--', label=f'Lower Threshold ({LOWER_THRESHOLD}°)')
    
    # 카운팅 이벤트 표시
    for frame_idx, angle, state in count_events:
        color = 'red' if state == 'DOWN' else 'green'
        ax.plot(frame_idx, angle, 'o', color=color, markersize=8, markeredgecolor='black', alpha=0.8)
    
    ax.set_xlabel("Frame Number")
    ax.set_ylabel("Angle (Degrees)")
    ax.set_ylim(0, 180)
    ax.legend(loc='upper right')
    ax.grid(True, linestyle='--', alpha=0.7)

    logging.info(f"Final Repetition Count for {sample_name}: {rep_count}")
    plt.tight_layout()
    plt.show()

# --- 3. 데이터프레임 순회 및 플롯 실행 (경로 기반 필터링) ---

TARGET_ACTIVITY_NAME = 'frontal__biceps_curl' 
TARGET_ACTIVITY_PATH_PATTERN = f'/{TARGET_ACTIVITY_NAME}'

target_df = train_df[train_df['interp_json_path'].str.contains(TARGET_ACTIVITY_PATH_PATTERN, case=False, na=False)].copy()

logging.info("-" * 50)
logging.info(f"Plotting Threshold Counter for activity: {TARGET_ACTIVITY_NAME} (Found {len(target_df)} samples)")

NUM_SAMPLES_TO_PLOT = 1 

if len(target_df) == 0:
    logging.warning(f"No samples found for activity: {TARGET_ACTIVITY_NAME}")
else:
    for idx, row in target_df.head(NUM_SAMPLES_TO_PLOT).iterrows():
        json_dir = row['interp_json_path']
        sample_name = Path(json_dir).name
        
        logging.info(f"Analyzing sample: {sample_name}...")
        
        keypoints_sequence = load_keypoints_from_directory(json_dir)
        
        # 🚨 Threshold 카운터 함수 호출
        plot_and_count_with_threshold(keypoints_sequence, sample_name)

logging.info("-" * 50)
logging.info("Analysis complete.")


INFO - Total training samples: 83
INFO - --------------------------------------------------
INFO - Plotting Threshold Counter for activity: frontal__biceps_curl (Found 3 samples)
INFO - Analyzing sample: frontal__biceps_curl...
INFO -   Loaded 0 frames from frontal__biceps_curl
WARNING - Cannot count: keypoints sequence is empty or invalid for frontal__biceps_curl
INFO - --------------------------------------------------
INFO - Analysis complete.


## 속도 기반

In [7]:
import pandas as pd
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import logging
from scipy.ndimage import uniform_filter1d # 노이즈 제거를 위한 필터

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')

# --- 0. 각도 계산 유틸리티 함수 (이전과 동일) ---

def calculate_angle(p1, p2, p3):
    """세 점 p1, p2(center), p3을 이용해 각도를 계산합니다."""
    try:
        p1 = np.array(p1)
        p2 = np.array(p2)
        p3 = np.array(p3)
        vec_a = p1 - p2
        vec_c = p3 - p2
        dot_product = np.dot(vec_a, vec_c)
        norm_a = np.linalg.norm(vec_a)
        norm_c = np.linalg.norm(vec_c)
        
        if norm_a == 0 or norm_c == 0:
            return np.nan
        
        cosine_angle = np.clip(dot_product / (norm_a * norm_c), -1.0, 1.0)
        angle_deg = np.degrees(np.arccos(cosine_angle))
        
        return angle_deg
    except Exception:
        return np.nan

# 분석할 팔꿈치 각도 정의
ANGLE_DEFINITIONS = {
    "L_Elbow(5-7-9)": (5, 7, 9),      # Shoulder - Elbow - Wrist
}

# --- 1. 데이터 로드 함수 (이전과 동일) ---

df = pd.read_csv("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_Repeatition_Counter/data/metadata.csv")
train_df = df[df['is_train'] == True].copy()
logging.info(f"Total training samples: {len(train_df)}")

def load_keypoints_from_directory(json_dir):
    # ... (이전 코드와 동일) ...
    try:
        json_path = Path(json_dir)
        # ... (중략) ...
        json_files = sorted([f for f in json_path.glob('*.json')])
        # ... (중략) ...
        
        keypoints_sequence = []
        for json_file in json_files:
            try:
                with open(json_file, 'r') as f:
                    frame_data = json.load(f)
                keypoints_sequence.append(frame_data)
            except Exception as e:
                logging.error(f"Error loading {json_file}: {e}")
                keypoints_sequence.append(None)
        
        logging.info(f"  Loaded {len(keypoints_sequence)} frames from {json_path.name}")
        return keypoints_sequence
        
    except Exception as e:
        logging.error(f"Error loading from {json_dir}: {e}")
        return None

# --- 2. 변화율 기반 카운팅 및 시각화 함수 ---

def plot_and_count_with_derivative(keypoints_sequence, sample_name):
    
    if not keypoints_sequence or not keypoints_sequence[0]:
        logging.warning(f"Cannot count: keypoints sequence is empty or invalid for {sample_name}")
        return
    
    # 2-1. L_Elbow 각도 추출 (이전과 동일)
    # ... (생략: 각도 추출 로직) ...
    angle_trajectories = {name: [] for name in ANGLE_DEFINITIONS.keys()}
    for frame_data in keypoints_sequence:
        current_kps = {}
        if (frame_data and 'instance_info' in frame_data and 
            len(frame_data['instance_info']) > 0 and 
            'keypoints' in frame_data['instance_info'][0]):
            
            keypoints_list = frame_data['instance_info'][0]['keypoints']
            keypoints_array = np.array(keypoints_list)
            for idx in range(len(keypoints_array)):
                current_kps[idx] = keypoints_array[idx, :2] 

        for angle_name, (p1_idx, p2_idx, p3_idx) in ANGLE_DEFINITIONS.items():
            if all(idx in current_kps for idx in (p1_idx, p2_idx, p3_idx)):
                angle = calculate_angle(current_kps[p1_idx], current_kps[p2_idx], current_kps[p3_idx])
                angle_trajectories[angle_name].append(angle)
            else:
                angle_trajectories[angle_name].append(np.nan)

    if 'L_Elbow(5-7-9)' not in angle_trajectories:
        logging.warning("L_Elbow data not found in trajectories.")
        return
        
    raw_angles = np.array(angle_trajectories['L_Elbow(5-7-9)'])
    
    # NaN 값 처리: 결측치는 이전 유효 값으로 채우거나 0으로 설정 (필터링을 위해)
    angles_filled = np.nan_to_num(raw_angles, nan=np.nanmean(raw_angles)) if np.nanmean(raw_angles) else np.nan_to_num(raw_angles, nan=0)
    
    # 2-2. 데이터 스무딩 (Moving Average Filter)
    window_size = 7 # 윈도우 크기를 약간 늘려 미분 시의 노이즈 영향을 줄입니다.
    smoothed_angles = uniform_filter1d(angles_filled, size=window_size, mode='nearest')
    
    # 2-3. 변화율 (Derivative) 계산
    # np.diff는 배열의 길이보다 1 작은 배열을 반환합니다.
    derivative = np.diff(smoothed_angles)
    
    # 2-4. 제로 크로싱 기반 카운팅
    
    # 바이셉스 컬 (팔꿈치 각도 기준):
    # 수축(올라가는 구간): 각도가 감소 (Derivative < 0)
    # 이완(내려가는 구간): 각도가 증가 (Derivative > 0)
    
    # 🚨 카운팅 임계값 (Min Velocity Threshold): 움직임으로 인정할 최소 변화율
    # 정지 상태의 작은 노이즈(Jitter)를 무시하기 위해 설정합니다.
    MIN_VELOCITY_THRESHOLD = 0.5 # 예: 0.5 deg/frame
    
    rep_count = 0
    # 상태: 0=정지, 1=수축, 2=이완
    current_state = 0 
    
    # 카운팅 이벤트 시각화를 위한 리스트
    count_events = []

    # derivative 배열은 smoothed_angles보다 1 작습니다.
    for i in range(len(derivative)):
        velocity = derivative[i]
        
        if current_state == 0: # 정지 또는 이전 상태를 모르는 경우
            if velocity < -MIN_VELOCITY_THRESHOLD:
                current_state = 1 # 수축 시작 (각도 감소)
            elif velocity > MIN_VELOCITY_THRESHOLD:
                current_state = 2 # 이완 시작 (각도 증가)
        
        elif current_state == 1: # 수축 중 (각도 감소 중)
            # 속도가 양수 임계값을 넘어가면 (방향 전환) -> 이완 시작
            if velocity > MIN_VELOCITY_THRESHOLD:
                rep_count += 1
                current_state = 2 # 이완으로 전환
                count_events.append((i + 1, smoothed_angles[i + 1], 'REP_END')) # Rep 완료 시점
        
        elif current_state == 2: # 이완 중 (각도 증가 중)
            # 속도가 음수 임계값을 넘어가면 (방향 전환) -> 수축 시작
            if velocity < -MIN_VELOCITY_THRESHOLD:
                current_state = 1 # 수축으로 전환
                count_events.append((i + 1, smoothed_angles[i + 1], 'REP_START')) # 다음 Rep 시작 시점

    
    # 2-5. 시각화
    frames = np.arange(len(smoothed_angles))
    fig, axes = plt.subplots(2, 1, figsize=(18, 10), sharex=True)
    
    fig.suptitle(f"Derivative-based Repetition Counter for L_Elbow (Sample: {sample_name})\nTotal Reps: {rep_count}", fontsize=16)

    # 상단 플롯: 스무딩된 각도 및 카운팅 이벤트
    ax_angle = axes[0]
    ax_angle.plot(frames, smoothed_angles, label='Smoothed L_Elbow Angle', color='blue', linewidth=2)
    ax_angle.set_title("Smoothed Angle Trajectory")
    ax_angle.set_ylabel("Angle (Degrees)")
    ax_angle.set_ylim(0, 180)
    
    # 카운팅 이벤트 표시
    for frame_idx, angle, event_type in count_events:
        color = 'green' if event_type == 'REP_END' else 'red'
        ax_angle.plot(frame_idx, angle, 'o', color=color, markersize=8, alpha=0.8, label=event_type if frame_idx == count_events[0][0] else "")
    ax_angle.legend(loc='upper right')
    ax_angle.grid(True, linestyle='--', alpha=0.7)

    # 하단 플롯: 변화율 (속도)
    ax_deriv = axes[1]
    # derivative는 길이가 1 짧으므로 인덱스를 조정하여 플롯합니다.
    ax_deriv.plot(frames[:-1], derivative, label='Angle Velocity (Derivative)', color='darkorange', linewidth=1.5)
    
    # 속도 임계값 표시 (노이즈 방지 영역)
    ax_deriv.axhline(MIN_VELOCITY_THRESHOLD, color='gray', linestyle=':', label='Velocity Threshold')
    ax_deriv.axhline(-MIN_VELOCITY_THRESHOLD, color='gray', linestyle=':')
    ax_deriv.axhline(0, color='black', linestyle='-', linewidth=0.5)

    ax_deriv.set_title("Angle Velocity (Change per Frame)")
    ax_deriv.set_xlabel("Frame Number")
    ax_deriv.set_ylabel("Velocity (Deg/Frame)")
    ax_deriv.legend(loc='upper right')
    ax_deriv.grid(True, linestyle='--', alpha=0.7)

    logging.info(f"Final Repetition Count for {sample_name}: {rep_count}")
    plt.tight_layout()
    plt.show()

# --- 3. 데이터프레임 순회 및 플롯 실행 ---

TARGET_ACTIVITY_NAME = 'frontal__biceps_curl' 
TARGET_ACTIVITY_PATH_PATTERN = f'/{TARGET_ACTIVITY_NAME}'

target_df = train_df[train_df['interp_json_path'].str.contains(TARGET_ACTIVITY_PATH_PATTERN, case=False, na=False)].copy()

logging.info("-" * 50)
logging.info(f"Plotting Derivative Counter for activity: {TARGET_ACTIVITY_NAME} (Found {len(target_df)} samples)")

NUM_SAMPLES_TO_PLOT = 1 

if len(target_df) == 0:
    logging.warning(f"No samples found for activity: {TARGET_ACTIVITY_NAME}")
else:
    for idx, row in target_df.head(NUM_SAMPLES_TO_PLOT).iterrows():
        json_dir = row['interp_json_path']
        sample_name = Path(json_dir).name
        
        logging.info(f"Analyzing sample: {sample_name}...")
        
        keypoints_sequence = load_keypoints_from_directory(json_dir)
        
        # 🚨 변화율 카운터 함수 호출
        plot_and_count_with_derivative(keypoints_sequence, sample_name)

logging.info("-" * 50)
logging.info("Analysis complete.")

INFO - Total training samples: 83
INFO - --------------------------------------------------
INFO - Plotting Derivative Counter for activity: frontal__biceps_curl (Found 3 samples)
INFO - Analyzing sample: frontal__biceps_curl...
INFO -   Loaded 0 frames from frontal__biceps_curl
WARNING - Cannot count: keypoints sequence is empty or invalid for frontal__biceps_curl
INFO - --------------------------------------------------
INFO - Analysis complete.
